In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
# ========== python ==========
import os
from pathlib import Path
from logging import Logger
from typing import List, Dict, Tuple, Optional, Union, Callable, Final, Literal, get_args
from operator import itemgetter, attrgetter
import itertools
from IPython.display import display

from utils.setup import setup_logger, get_device
from const.const_values import PROJECT_DIR

os.chdir(PROJECT_DIR)
logger: Logger = setup_logger(__name__, f'{PROJECT_DIR}/log/jupyter_run.log')
device = get_device(device_name='cpu', logger=logger)

In [2]:
# jupyter
import seaborn as sns
import matplotlib.pyplot as plt
# Machine learning
import numpy as np
import pandas as pd
import h5py
import optuna
# torch
import torch
from torch.nn import CrossEntropyLoss
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
from torch.utils.tensorboard.writer import SummaryWriter
# torch ignite
from ignite.engine import Engine
from ignite.handlers import Checkpoint
# My items
from models.datasets.data_helper import MyDataHelper, MyDataLoaderHelper, DefaultTokens
from models.datasets.datasets_for_story import StoryTriple
# My utils
from utils.setup import load_param
from utils.torch import load_model, torch_fix_seed
# main function
from run_for_KGC import main_function


In [3]:
from const.const_values import CPU, MODEL
from models.KGModel.kg_model import HEAD, RELATION, TAIL
from utils.torch_ignite import TRAINER, EVALUATOR
from const.const_values import DATASETS, DATA_HELPER, DATA_LOADERS, TRAIN_RETURNS

In [4]:
SEED: Final[int] = 42
args_path = f'{PROJECT_DIR}/models/kgc03/DevilsFoot075/param.pkl'
model_path = f'{PROJECT_DIR}/models/kgc03/DevilsFoot075/model.pth'

In [5]:
args = load_param(args_path)

# args.pre_train = True
args.logger = logger
args.device = device
args.batch_size = 1
args.pre_train=False
args.init_embedding_using_bert = False
args.model_path = model_path
args.only_load_trainer_evaluator = True

del args.optuna_file, args.device_name, args.pid, args.study_name, args.n_trials
logger.info(args)

2023-02-04 06:44:19 - INFO - 2517384385.py - 13 - Namespace(notebook=False, console_level='info', logfile='models/kgc03/DevilsFoot075/log.log', param_file='models/kgc03/DevilsFoot075/param.pkl', train_anyway=False, old_data=0, tensorboard_dir='models/kgc03/DevilsFoot075/tensorboard', checkpoint_dir='models/kgc03/DevilsFoot075/checkpoint/', model_path='/Users/ryoyakaneda/Documents/学校/M1Study/knowledge_graph/models/kgc03/DevilsFoot075/model.pth', resume_from_checkpoint=False, resume_from_last_point=False, only_load_trainer_evaluator=True, resume_checkpoint_path=None, pre_train=False, train_valid_test=False, only_train=True, use_for_challenge100=False, use_for_challenge090=False, use_for_challenge075=True, use_title='DevilsFoot', do_optuna=False, story_special_num=5, relation_special_num=5, entity_special_num=5, padding_token_e=0, cls_token_e=1, mask_token_e=2, sep_token_e=3, bos_token_e=4, padding_token_r=0, cls_token_r=1, mask_token_r=2, sep_token_r=3, bos_token_r=4, padding_token_s=0, 

In [6]:
torch_fix_seed(seed=SEED)
return_dict = main_function(args, logger=logger)

2023-02-04 06:44:19 - INFO - run_for_KGC.py - 893 - ----- make datahelper start. -----
2023-02-04 06:44:20 - INFO - data_helper.py - 333 - ====== Show MyRawData ======
2023-02-04 06:44:20 - INFO - data_helper.py - 334 - entity num: 7812
2023-02-04 06:44:20 - INFO - data_helper.py - 335 - relation num: 62
2023-02-04 06:44:20 - INFO - data_helper.py - 336 - ====== Show MyRawData ======
2023-02-04 06:44:20 - INFO - data_helper.py - 607 - ========== Show DataHelper ==========
2023-02-04 06:44:20 - INFO - data_helper.py - 333 - ====== Show MyRawData ======
2023-02-04 06:44:20 - INFO - data_helper.py - 334 - entity num: 7812
2023-02-04 06:44:20 - INFO - data_helper.py - 335 - relation num: 62
2023-02-04 06:44:20 - INFO - data_helper.py - 336 - ====== Show MyRawData ======
2023-02-04 06:44:20 - INFO - data_helper.py - 609 - entity_special_dicts: {0: '<pad_e>', 1: '<cls_e>', 2: '<mask_e>', 3: '<sep_e>', 4: '<bos_e>'}
2023-02-04 06:44:20 - INFO - data_helper.py - 610 - relation_special_dicts: {

In [7]:
model = return_dict[MODEL]

dataset_train: StoryTriple = return_dict[DATASETS][0]
triple: torch.Tensor = dataset_train.triple
data_helper: MyDataHelper = return_dict[DATA_HELPER]
evaluator: Checkpoint = return_dict[TRAIN_RETURNS][EVALUATOR]

load_model(model, args.model_path, device)
model.eval()

entities, relations = data_helper.processed_entities, data_helper.processed_relations
d_e, d_r = {e: i for i, e in enumerate(entities)}, {r: i for i, r in enumerate(relations)}

triple_df = pd.DataFrame([(entities[_t[0]], relations[_t[1]], entities[_t[2]]) for _t in triple], columns=[HEAD, RELATION, TAIL])
story_entities = triple_df[HEAD].tolist()

In [8]:
triple_df

,head,relation,tail
0,<bos_e>,<bos_r>,<bos_e>
1,AbbeyGrange:001,kgc:subject,AllTitle:Holmes
2,AbbeyGrange:001,kgc:hasPredicate,word.predicate:wakeUp
3,AbbeyGrange:001,kgc:whom,AllTitle:Watson
4,AbbeyGrange:001,kgc:infoSource,AllTitle:Watson
...,...,...,...
17183,AbbeyGrange:084,kgc:subject,AbbeyGrange:Theresa
17184,AbbeyGrange:084,kgc:hasPredicate,word.predicate:rest
17185,AbbeyGrange:084,kgc:where,AbbeyGrange:room_of_Lady_Brackenstall
17186,<bos_e>,<bos_r>,<bos_e>


In [9]:
def extract(_model, target, inputs):
    features = None

    def forward_hook(_module, _inputs, _):
        nonlocal features
        x, _, _ = _inputs
        outputs = _module.forward(x, x, x, need_weights=True)[1]
        features = outputs.detach().clone()

    handle = target.register_forward_hook(forward_hook)

    _model.eval()
    _model(inputs, torch.LongTensor([[]]), torch.LongTensor([[]]), torch.LongTensor([[]]))

    handle.remove()

    return features

def get_attention(input_):
    assert len(input_) == 1
    features = extract(model, model.transformer.layers[-1].self_attn, input_)[0]
    df_attention = pd.DataFrame([[entities[h], relations[r], entities[t]]+[features[j, i].item() for j in range(len(features))] for i, (h, r, t) in enumerate(input_[0])])
    df_attention.columns=[HEAD, RELATION, TAIL] + [f'atten_from{i}' for i in range(len(df_attention.columns)-3)]
    return df_attention

def show_attension_heatmap(df_attention):
    sns.heatmap(df_attention.iloc[:,3:])
    plt.show()

In [10]:
MASK_E = DefaultTokens.MASK_E
KILL = 'word.predicate:kill'

TAKE = 'word.predicate:take'
BRING = 'word.predicate:bring'
DIE = 'word.predicate:die'
HIDE = 'word.predicate:hide'

In [11]:
bos_triple = [d_e[DefaultTokens.BOS_E], d_r[DefaultTokens.BOS_R],d_e[DefaultTokens.BOS_E]]
mask_e_id = d_e[DefaultTokens.MASK_E]
Holmes_id =d_e['AllTitle:Holmes']

def make_ranking(from_story_name, to_story_name, predicate_, whom_, subject_, why_, what_, where_):
    if not (from_story_name is None and to_story_name is None):
        _start_index = story_entities.index(from_story_name)-1 
        _end_index = len(story_entities) - story_entities[::-1].index(to_story_name)
    else: 
        _start_index = 0
        _end_index = 0
    question_ = torch.tensor(
        [
            bos_triple, 
            [mask_e_id, d_r['kgc:infoSource'],     Holmes_id      ],
            [mask_e_id, d_r['kgc:hasPredicate'],   d_e[predicate_]],
            [mask_e_id, d_r['kgc:whom'],           d_e[whom_     ]],
            [mask_e_id, d_r['kgc:subject'],        d_e[subject_  ]],
            [mask_e_id, d_r['kgc:why'],            d_e[why_      ]],
            [mask_e_id, d_r['kgc:what'],           d_e[what_     ]],
            [mask_e_id, d_r['kgc:where'],          d_e[where_    ]],
        ]
    )
    mask_ = torch.zeros_like(question_, dtype=torch.bool) # not mask all position
    mask_[1:, 0] = True                                   # where head position without bos token
    mask_[1:, 2] = True                                   # where tail position without bos token

    last_triples = triple[_start_index: _end_index]

    questions = torch.cat([last_triples, question_], dim=0).unsqueeze(0)
    masks = torch.cat([torch.zeros_like(last_triples), mask_], dim=0).to(torch.bool).transpose(1,0).unsqueeze(0)

    data_list = []
    with torch.no_grad():
        _, (story_pred, relation_pred, entity_pred) = model(questions, masks[:,0], masks[:,1], masks[:,2])
        sorted_ = torch.argsort(entity_pred, dim=1, descending=True)
        for i in range(sorted_.shape[1]):
            ans_= sorted_[:, i]
            info_source_, predicate_pred, whom_pred, subject_pred, why_pred, what_pred, where_pred = ans_
            data_list.append([entities[predicate_pred], entities[whom_pred], entities[subject_pred], entities[why_pred], entities[what_pred], entities[where_pred]])
    df_ranking = pd.DataFrame(data_list, columns=['predicate', 'whom', 'subject', 'why', 'what', 'where'])
    df_attension = get_attention(questions)
    
    return df_ranking, df_attension

In [12]:
def main_func01(_title, _victim_name, criminal, predicate, _last_index, _story_len):
    from_ = f'{_title}:{_last_index-_story_len+1}'
    to_ = f'{_title}:{_last_index}'
    predicate = predicate
    victim = f'{_title}:{_victim_name}'
    criminal = f'{_title}:{criminal}'
    df_ranking, df_attension = make_ranking(
        from_, to_, predicate, victim, MASK_E, MASK_E, MASK_E, MASK_E)

    pred_rank = df_ranking.index[df_ranking['subject']==criminal].tolist()
    pred_rank = pred_rank[0] if len(pred_rank)==1 else -1
    logger.info(f"The pred ranking about {criminal} is {pred_rank}")
    display(df_ranking.iloc[:max(20, pred_rank)])
    len_ = len(df_attension)
    for i in range(len_-10, len_):
        print(f"index={i}, triple={df_attension.iloc[i,:3].tolist()}, attention list")
        display(df_attension.sort_values(f'atten_from{i}', ascending=False).iloc[:,[0,1,2,3+i]],)
    return df_ranking, df_attension

def check_killer(_title, _victim_name, _killer_name, _last_index, _story_len):
    return main_func01(_title, _victim_name, _killer_name, KILL, _last_index, _story_len)

In [13]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)

### DevilsFoot(悪魔の足跡１)
Who killed the victims? (criminal & explanation)
被害者: Brenda
犯人: Mortimer
犯行動機: 財産

In [17]:
def do_devil1_pred():
    title = 'DevilsFoot'
    victim_name = 'Brenda'
    killer_name = 'Mortimer'
    last_index = 366
    story_len = 80
    df_ranking, df_attention = check_killer(title, victim_name, killer_name, last_index, story_len)
    return df_ranking, df_attention

do_devil1_pred()
pass

2023-02-04 06:45:31 - INFO - 1707532137.py - 12 - The pred ranking about DevilsFoot:Mortimer is 0


,predicate,whom,subject,why,what,where
0,word.predicate:enter,DevilsFoot:Owen,DevilsFoot:Mortimer,ACaseOfIdentity:Sutherland,ACaseOfIdentity:Sutherland,<bos_e>
1,word.predicate:meet,DevilsFoot:Brenda,DevilsFoot:Owen,DevilsFoot:Owen,DevilsFoot:Owen,DevilsFoot:Mortimer
2,word.predicate:exist,DevilsFoot:George,DevilsFoot:George,AbbeyGrange:Lady_Brackenstall,DevilsFoot:Mortimer,DevilsFoot:living_room
3,word.predicate:say,DevilsFoot:Mortimer,DevilsFoot:Brenda,DevilsFoot:Brenda,AllTitle:Watson,SpeckledBand:1883-04-01T14
4,word.predicate:think,AllTitle:Holmes,AllTitle:Holmes,AllTitle:Watson,word.predicate:say,DevilsFoot:Roundhay
5,word.predicate:visit,AllTitle:Watson,AllTitle:Watson,ACaseOfIdentity:Windibank,DevilsFoot:Brenda,DancingMen:Abe_Slaney
6,word.predicate:love,ACaseOfIdentity:Sutherland,ResidentPatient:Percy_Trevelyan,DevilsFoot:Mortimer,ACaseOfIdentity:mother_of_Sutherland,SpeckledBand:1883-04-02T04
7,word.predicate:kill,SpeckledBand:Roylott,ACaseOfIdentity:Sutherland,DateTime:1898-08-13T03:00:00,DancingMen:Abe_Slaney,SpeckledBand:the_day_Julia_died
8,word.predicate:want,ACaseOfIdentity:Hosmer,ACaseOfIdentity:Hosmer,word.predicate:say,ACaseOfIdentity:Windibank,word.predicate:have
9,word.predicate:try,AbbeyGrange:Lady_Brackenstall,word.predicate:say,DevilsFoot:Roundhay,DevilsFoot:Case_1,SpeckledBand:Roylott_s_bedroom


index=425, triple=['DevilsFoot:366', 'kgc:hasProperty', 'word.predicate:suicide'], attention list


,head,relation,tail,atten_from425
424,DevilsFoot:366,kgc:subject,DevilsFoot:reason_of_death_of_Mortimer,0.010348
434,<mask_e>,kgc:where,<mask_e>,0.009263
425,DevilsFoot:366,kgc:hasProperty,word.predicate:suicide,0.009132
433,<mask_e>,kgc:what,<mask_e>,0.007555
428,<mask_e>,kgc:infoSource,AllTitle:Holmes,0.007359
426,DevilsFoot:366,kgc:infoSource,DevilsFoot:Watson_s_remarks,0.007092
432,<mask_e>,kgc:why,<mask_e>,0.006217
431,<mask_e>,kgc:subject,<mask_e>,0.006015
386,DevilsFoot:360,kgc:subject,DevilsFoot:Case_1,0.005971
326,DevilsFoot:348,kgc:what,DevilsFoot:355,0.005509


index=426, triple=['DevilsFoot:366', 'kgc:infoSource', 'DevilsFoot:Watson_s_remarks'], attention list


...

,head,relation,tail,atten_from432
428,<mask_e>,kgc:infoSource,AllTitle:Holmes,0.012680
351,DevilsFoot:352,kgc:infoSource,AllTitle:Holmes,0.011436
343,DevilsFoot:350,kgc:infoSource,AllTitle:Holmes,0.010712
333,DevilsFoot:348,kgc:what,DevilsFoot:362,0.010552
324,DevilsFoot:348,kgc:what,DevilsFoot:353,0.010493
326,DevilsFoot:348,kgc:what,DevilsFoot:355,0.010276
323,DevilsFoot:348,kgc:what,DevilsFoot:352,0.009978
334,DevilsFoot:348,kgc:what,DevilsFoot:363,0.009741
380,DevilsFoot:359,kgc:subject,DevilsFoot:Mortimer,0.009585
325,DevilsFoot:348,kgc:what,DevilsFoot:354,0.009390


index=433, triple=['<mask_e>', 'kgc:what', '<mask_e>'], attention list


,head,relation,tail,atten_from433
428,<mask_e>,kgc:infoSource,AllTitle:Holmes,0.014824
333,DevilsFoot:348,kgc:what,DevilsFoot:362,0.013938
351,DevilsFoot:352,kgc:infoSource,AllTitle:Holmes,0.011335
414,DevilsFoot:364,kgc:infoSource,AllTitle:Holmes,0.011330
323,DevilsFoot:348,kgc:what,DevilsFoot:352,0.011306
334,DevilsFoot:348,kgc:what,DevilsFoot:363,0.011054
326,DevilsFoot:348,kgc:what,DevilsFoot:355,0.010877
324,DevilsFoot:348,kgc:what,DevilsFoot:353,0.010852
343,DevilsFoot:350,kgc:infoSource,AllTitle:Holmes,0.010638
325,DevilsFoot:348,kgc:what,DevilsFoot:354,0.010069


index=434, triple=['<mask_e>', 'kgc:where', '<mask_e>'], attention list


,head,relation,tail,atten_from434
357,DevilsFoot:354,kgc:subject,DevilsFoot:Mortimer,0.015565
345,DevilsFoot:351,kgc:subject,DevilsFoot:Mortimer,0.015294
333,DevilsFoot:348,kgc:what,DevilsFoot:362,0.012187
380,DevilsFoot:359,kgc:subject,DevilsFoot:Mortimer,0.012105
341,DevilsFoot:350,kgc:subject,DevilsFoot:Mortimer,0.012044
323,DevilsFoot:348,kgc:what,DevilsFoot:352,0.011586
334,DevilsFoot:348,kgc:what,DevilsFoot:363,0.011409
86,DevilsFoot:302a,kgc:subject,DevilsFoot:Case_1,0.010612
370,DevilsFoot:357,kgc:subject,DevilsFoot:Mortimer,0.010549
321,DevilsFoot:348,kgc:what,DevilsFoot:350,0.010386


### DevilsFoot(悪魔の足跡2)
Who killed the victims? (criminal & explanation)
被害者: Mortimer
犯人: 
犯行動機: 恋人の敵

In [18]:
def do_devil2_pred():
    title = 'DevilsFoot'
    victim_name = 'Mortimer'
    killer_name = 'Sterndale'
    last_index = 366
    story_len = 80
    df_ranking, df_attention = check_killer(title, victim_name, killer_name, last_index, story_len)
    return df_ranking, df_attention

do_devil2_pred()
pass

2023-02-04 06:45:37 - INFO - 1707532137.py - 12 - The pred ranking about DevilsFoot:Sterndale is 154


,predicate,whom,subject,why,what,where
0,word.predicate:enter,DevilsFoot:Mortimer,DevilsFoot:Mortimer,ACaseOfIdentity:Sutherland,AllTitle:Watson,DevilsFoot:Mortimer
1,word.predicate:meet,DevilsFoot:Owen,DevilsFoot:Owen,DevilsFoot:Owen,DevilsFoot:Mortimer,DevilsFoot:living_room
2,word.predicate:think,DevilsFoot:George,AllTitle:Watson,AbbeyGrange:Lady_Brackenstall,ACaseOfIdentity:Sutherland,<bos_e>
3,word.predicate:visit,AllTitle:Watson,DevilsFoot:George,AllTitle:Watson,DevilsFoot:Owen,SpeckledBand:1883-04-01T14
4,word.predicate:say,DevilsFoot:Brenda,AllTitle:Holmes,DevilsFoot:Brenda,ACaseOfIdentity:mother_of_Sutherland,DevilsFoot:Roundhay
5,word.predicate:die,AllTitle:Holmes,DevilsFoot:Brenda,DateTime:1898-08-13T03:00:00,word.predicate:say,SpeckledBand:1883-04-02T04
6,word.predicate:angry,ACaseOfIdentity:Sutherland,ACaseOfIdentity:Sutherland,DevilsFoot:Mortimer,word.predicate:feel,word.predicate:have
7,word.predicate:exist,ACaseOfIdentity:Hosmer,ResidentPatient:Percy_Trevelyan,DevilsFoot:Roundhay,AllTitle:Holmes,AbbeyGrange:Sir_Eustace_Brackenstall
8,word.predicate:want,ResidentPatient:Percy_Trevelyan,DevilsFoot:Doctor_Richard,DevilsFoot:table,SilverBlaze:Ned_Hunter,SpeckledBand:the_day_Julia_died
9,word.predicate:kill,word.predicate:say,word.predicate:say,SpeckledBand:Roylott_s_bedroom,DevilsFoot:Brenda,DancingMen:Abe_Slaney


index=425, triple=['DevilsFoot:366', 'kgc:hasProperty', 'word.predicate:suicide'], attention list


,head,relation,tail,atten_from425
424,DevilsFoot:366,kgc:subject,DevilsFoot:reason_of_death_of_Mortimer,0.009940
425,DevilsFoot:366,kgc:hasProperty,word.predicate:suicide,0.007988
434,<mask_e>,kgc:where,<mask_e>,0.007309
428,<mask_e>,kgc:infoSource,AllTitle:Holmes,0.007138
426,DevilsFoot:366,kgc:infoSource,DevilsFoot:Watson_s_remarks,0.006547
431,<mask_e>,kgc:subject,<mask_e>,0.006540
433,<mask_e>,kgc:what,<mask_e>,0.006201
432,<mask_e>,kgc:why,<mask_e>,0.006084
386,DevilsFoot:360,kgc:subject,DevilsFoot:Case_1,0.006075
430,<mask_e>,kgc:whom,DevilsFoot:Mortimer,0.005553


index=426, triple=['DevilsFoot:366', 'kgc:infoSource', 'DevilsFoot:Watson_s_remarks'], attention list


...

,head,relation,tail,atten_from432
428,<mask_e>,kgc:infoSource,AllTitle:Holmes,0.012130
333,DevilsFoot:348,kgc:what,DevilsFoot:362,0.011845
326,DevilsFoot:348,kgc:what,DevilsFoot:355,0.011612
324,DevilsFoot:348,kgc:what,DevilsFoot:353,0.011556
323,DevilsFoot:348,kgc:what,DevilsFoot:352,0.011277
334,DevilsFoot:348,kgc:what,DevilsFoot:363,0.010870
380,DevilsFoot:359,kgc:subject,DevilsFoot:Mortimer,0.010506
325,DevilsFoot:348,kgc:what,DevilsFoot:354,0.010306
351,DevilsFoot:352,kgc:infoSource,AllTitle:Holmes,0.010178
431,<mask_e>,kgc:subject,<mask_e>,0.009786


index=433, triple=['<mask_e>', 'kgc:what', '<mask_e>'], attention list


,head,relation,tail,atten_from433
333,DevilsFoot:348,kgc:what,DevilsFoot:362,0.013548
428,<mask_e>,kgc:infoSource,AllTitle:Holmes,0.013450
323,DevilsFoot:348,kgc:what,DevilsFoot:352,0.011259
326,DevilsFoot:348,kgc:what,DevilsFoot:355,0.010739
334,DevilsFoot:348,kgc:what,DevilsFoot:363,0.010687
324,DevilsFoot:348,kgc:what,DevilsFoot:353,0.010497
351,DevilsFoot:352,kgc:infoSource,AllTitle:Holmes,0.010096
325,DevilsFoot:348,kgc:what,DevilsFoot:354,0.009879
321,DevilsFoot:348,kgc:what,DevilsFoot:350,0.009855
414,DevilsFoot:364,kgc:infoSource,AllTitle:Holmes,0.009587


index=434, triple=['<mask_e>', 'kgc:where', '<mask_e>'], attention list


,head,relation,tail,atten_from434
357,DevilsFoot:354,kgc:subject,DevilsFoot:Mortimer,0.020018
345,DevilsFoot:351,kgc:subject,DevilsFoot:Mortimer,0.015623
341,DevilsFoot:350,kgc:subject,DevilsFoot:Mortimer,0.014506
380,DevilsFoot:359,kgc:subject,DevilsFoot:Mortimer,0.013032
361,DevilsFoot:355,kgc:subject,DevilsFoot:Mortimer,0.012852
365,DevilsFoot:356,kgc:subject,DevilsFoot:something,0.011699
353,DevilsFoot:353,kgc:subject,DevilsFoot:Mortimer,0.011651
323,DevilsFoot:348,kgc:what,DevilsFoot:352,0.011566
370,DevilsFoot:357,kgc:subject,DevilsFoot:Mortimer,0.011523
333,DevilsFoot:348,kgc:what,DevilsFoot:362,0.011395
